# Amazon Bedrock Knowledge Bases - 여러 데이터 소스를 사용하는 엔드 투 엔드 예제

이 노트북은 Amazon Bedrock Knowledge Bases를 사용해 다양한 데이터 소스(S3, Confluence, SharePoint, Salesforce, Web)에서 문서를 색인에 적재하고 RAG 애플리케이션을 구축하는 엔드 투 엔드 예제를 위한 샘플 코드를 제공합니다. 데이터 소스는 최대 5개까지 추가할 수 있습니다.

#### 노트북 워크스루

여러 데이터 소스에 저장된 문서를 Knowledge Base(예: Amazon OpenSearch Service Serverless(AOSS)와 같은 벡터 데이터베이스)에 적재해, 질문이 들어왔을 때 조회할 수 있도록 하는 데이터 파이프라인을 구성합니다.

- 다양한 데이터 소스(S3, Confluence, SharePoint, Salesforce, Web)를 연결해 문서를 Knowledge Base에 적재합니다.
- Ingestion 동안 Knowledge Base는 선택한 전략에 따라 문서를 작은 청크로 분할하고, 임베딩을 생성해 연결된 벡터 스토어에 저장합니다.

<!-- ![data_ingestion.png](./images/data_ingestion.png) -->
<img src="./images/data_ingestion.png" width=50% height=20% />

#### 단계: 
- 다양한 데이터 소스(S3, Confluence, SharePoint, Salesforce, Web)에 접근하고 OSS에 임베딩을 기록하기 위한 권한을 포함한 Knowledge Base 실행 역할을 생성합니다.
- 비어 있는 OpenSearch Serverless 인덱스를 생성합니다.
- 사전 준비:
    - S3: 버킷이 없으면 생성하고 데이터를 업로드합니다.
    - 기타 데이터 소스: 해당 [AWS 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/data-source-connectors.html)의 사전 준비 사항을 참고합니다.
- Knowledge Base를 생성합니다.
- Knowledge Base 안에 데이터 소스를 생성합니다.
- 각 데이터 소스에 대해 KB API를 사용해 ingestion 작업을 시작합니다. 이 작업은 데이터 소스에서 데이터를 읽고, 청크로 분할하고, Amazon Titan Embeddings 모델로 임베딩을 생성한 뒤 AOSS에 저장합니다. 이 모든 과정을 데이터 파이프라인을 직접 구축·배포·운영하지 않고 수행할 수 있습니다.

데이터가 Bedrock Knowledge Base에 준비되면 Amazon Bedrock이 제공하는 Knowledge Base API를 사용해 질문 응답 애플리케이션을 구축할 수 있습니다.

<div class="alert alert-block alert-info">
<b>참고:</b> Amazon Bedrock 콘솔에서 `amazon.nova-micro-v1:0`, `Anthropic Claude 3 Sonnet`, `amazon.titan-text-express-v1`, `anthropic.claude-3-haiku-20240307-v1:0`, `Titan Text Embeddings V2` 모델 접근 권한을 활성화하세요.
<br> -------------------------------------------------------------------------------------------------------------------------------------------------------   </br>
노트북 셀을 하나씩 실행하고 "Run All Cells" 옵션은 사용하지 마세요.
</div>

## 설정
나머지 셀을 실행하기 전에 아래 셀을 먼저 실행해 필요한 라이브러리를 설치하고 Bedrock에 연결하세요.

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-knowledge-base-{suffix}"
knowledge_base_description = "Multi data source knowledge base."

bucket_name = f'{knowledge_base_name}-{account_id}'
intermediate_bucket_name = f'{knowledge_base_name}-intermediate-{account_id}'

In [ ]:
print(boto3.__version__)

### 이제 Knowledge Base에 여러 종류의 데이터 소스(S3, Confluence, SharePoint, Salesforce, Web Crawler)를 추가할 수 있습니다. 이 노트북에서는 다양한 데이터 소스를 사용해 Knowledge Base 생성을 테스트합니다.

각 데이터 소스에는 서로 다른 사전 준비 사항이 있을 수 있으므로 자세한 내용은 AWS 문서를 참고하세요.

In [8]:
# For this notebook, we'll create Knowledge Base with multiple data sources ( 1 S3 bucket, 1 confluence page, 1 Sharepoint site, 1 Salesforce site, 1 Web Crawler)

data_bucket_name = f'bedrock-kb-{suffix}-1' # replace it with your first bucket name.

## Below is a list of data sources including, 1 S3 buckets, 1 confluence, 1 Sharepoint, 1 Salesforce connectors
## Please uncomment the data sources that you want to add and update the placeholder values accordingly.

data_sources=[
                {"type": "S3", "bucket_name": data_bucket_name}, 
                
                # {"type": "CONFLUENCE", "hostUrl": "https://example.atlassian.net", "authType": "BASIC",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>"},

                # {"type": "SHAREPOINT", "tenantId": "888d0b57-69f1-4fb8-957f-e1f0bedf64de", "domain": "yourdomain",
                #   "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>",
                #  "siteUrls": ["https://yourdomain.sharepoint.com/sites/mysite"]
                # },

                # {"type": "SALESFORCE", "hostUrl": "https://company.salesforce.com/", "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>"
                # },

                # {"type": "WEB", "seedUrls": [{ "url": "https://www.examplesite.com"}],
                #  "inclusionFilters": ["https://www\.examplesite\.com/.*\.html"],
                #  "exclusionFilters": ["https://www\.examplesite\.com/contact-us\.html"]
                # }
            ]
                
pp = pprint.PrettyPrinter(indent=2)

## Knowledge Base 생성

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

### Knowledge Base에 적재할 데이터 다운로드
다음 데이터를 사용합니다:
 - 첫 번째 데이터 소스로 로컬 디렉터리에 저장된 합성 데이터

#### 데이터를 S3 버킷 데이터 소스로 업로드

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../synthetic_dataset", data_bucket_name)

### Ingestion 작업 시작
KB와 데이터 소스를 생성한 뒤 각 데이터 소스에 대해 ingestion 작업을 시작할 수 있습니다.
Ingestion 작업 동안 KB는 데이터 소스에서 문서를 가져와 텍스트를 추출하고, 지정한 청킹 크기에 따라 분할하며, 각 청크의 임베딩을 생성해 이번에는 OSS 벡터 데이터베이스에 저장합니다.

참고: 현재는 한 번에 하나의 ingestion 작업만 실행할 수 있습니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

### 2.2 Knowledge Base 테스트
Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용해 확인할 수 있습니다. 

#### Retrieve and Generate API로 Knowledge Base 테스트

먼저 retrieve and generate API로 Knowledge Base를 시험해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색하고 Bedrock의 Foundation Model로 최종 답변을 생성합니다.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

해당 질의의 정답(ground truth QA 쌍 기준)은 다음과 같습니다:
```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow.
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```

In [13]:
query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019?"

In [ ]:
foundation_model = "us.amazon.nova-pro-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}:{}:inference-profile/{}".format(region, account_id, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

Retrieve and Generate API를 사용하면 최종 응답을 바로 받게 되므로, 응답에 사용된 다양한 소스를 확인할 수 없습니다. 이제 Retrieve API로 Knowledge Base의 소스 정보를 확인해 보겠습니다.

#### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하다면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 직접 가져올 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": "How many new positions were opened across Amazon's fulfillment and delivery network?"
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

### 정리
아래 섹션의 주석을 해제한 뒤 실행해 모든 리소스를 삭제하세요.

In [ ]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
# knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)